https://justinmorganwilliams.medium.com/how-to-make-a-time-lapse-heat-map-with-folium-using-nyc-bike-share-data-1ccd2e32c2e3

In [1]:
import folium
from folium.plugins import HeatMap, HeatMapWithTime

In [2]:
import pandas as pd
import numpy as np
import datetime as datetime

In [3]:
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format('Bay Area BikeShare Station Map')
stations_map = folium.Map(location=[37.56236, -122.150876], 
                          tiles='cartodbpositron',
                          zoom_start=10)
stations_map.get_root().html.add_child(folium.Element(title_html));

In [15]:
stations_map

In [5]:
stations = pd.read_csv('../data/station_data.csv')
trips = pd.read_csv('../data/trip_data.csv',
                    parse_dates=['Start Date', 'End Date'], 
                    infer_datetime_format=True)
weather = pd.read_csv('../data/weather_data.csv')

In [6]:
'''
A note included with the data indicates that several stations were
relocated. Those include stations 23, 24, 49, 69 and 72, which
became 85, 86, 87, 88 and 89, respectively. Subsequently, 89 later
became 90, as well.
'''
moved_stations=[23, 24, 49, 69, 72]
new_stations1=[85, 86, 87, 88, 89]
new_stations2=[90]

In [7]:
replace_zip = list(zip(moved_stations, new_stations1))

for s1, s2 in replace_zip:
    trips.loc[trips["Start Station"]==s1, "Start Station"]=s2

trips.loc[trips["Start Station"]==89, "Start Station"]=90

for s1, s2 in replace_zip:
    trips.loc[trips["End Station"]==s1, "End Station"]=s2
    
trips.loc[trips["End Station"]==89, "End Station"]=90

In [8]:
trips['month'] = trips['Start Date'].dt.month
trips['day'] = trips['Start Date'].dt.dayofweek
trips['hour'] = trips['Start Date'].dt.hour

In [9]:
stations_lst = stations.Id.to_list()

In [10]:
coords_lst = list(zip(stations.Lat, stations.Long))

In [11]:
names = stations.Name.to_list()
station_names = list(zip(stations_lst, names))

In [12]:
coords = list(zip(coords_lst, station_names))

In [13]:
for point, station in coords: #
            marker = folium.Marker(location=point,
                                   popup=station[1],
                                   icon=folium.Icon(color='blue',icon='bicycle', prefix='fa'),
                                   tooltip=station[0])
            marker.add_to(stations_map)

In [14]:
stations_map

In [15]:
trips.head()

,Trip ID,Start Date,Start Station,End Date,End Station,Subscriber Type,month,day,hour
0,913460,2015-08-31 23:26:00,50,2015-08-31 23:39:00,70,Subscriber,8,0,23
1,913459,2015-08-31 23:11:00,31,2015-08-31 23:28:00,27,Subscriber,8,0,23
2,913455,2015-08-31 23:13:00,47,2015-08-31 23:18:00,64,Subscriber,8,0,23
3,913454,2015-08-31 23:10:00,10,2015-08-31 23:17:00,8,Subscriber,8,0,23
4,913453,2015-08-31 23:09:00,51,2015-08-31 23:22:00,60,Customer,8,0,23


In [17]:
statlat = stations[['Id', 'Lat', 'Long']]

In [18]:
tripslat = trips[['Start Date', 'Start Station', 'month', 'day', 'hour']].merge(statlat,
                                                                                left_on='Start Station',
                                                                                right_on='Id')

In [19]:
tripslat['count'] = 1
tripscount = pd.DataFrame(tripslat.groupby(['Start Station',
                                            'Lat',
                                            'Long'])
                          ['count'].sum().sort_values(ascending=False))

In [20]:
lst = tripscount.groupby(['Lat', 'Long']).sum().reset_index().values.tolist()

In [21]:
lst

[[37.329732, -121.901782, 4968.0],
 [37.330165, -121.885831, 495.0],
 [37.330698, -121.888979, 774.0],
 [37.331415, -121.8932, 562.0],
 [37.332692, -121.900084, 647.0],
 [37.332808, -121.883891, 475.0],
 [37.333798, -121.886943, 856.0],
 [37.333955, -121.877349, 494.0],
 [37.333988, -121.894902, 1958.0],
 [37.335885, -121.88566, 1099.0],
 [37.336721, -121.894074, 1418.0],
 [37.337391, -121.886995, 832.0],
 [37.339301, -121.889937, 839.0],
 [37.342725, -121.895617, 1120.0],
 [37.348742, -121.894715, 885.0],
 [37.352601, -121.905733, 534.0],
 [37.385956, -122.083678, 1230.0],
 [37.389218, -122.081896, 1583.0],
 [37.390277, -122.066553, 978.0],
 [37.394358, -122.076713, 3580.0],
 [37.400241, -122.099076, 501.0],
 [37.400443, -122.108338, 1069.0],
 [37.40694, -122.106758, 1058.0],
 [37.4256839, -122.1377775, 376.0],
 [37.429082, -122.142805, 400.0],
 [37.443988, -122.164759, 1116.0],
 [37.444521, -122.163093, 605.0],
 [37.448598, -122.159504, 576.0],
 [37.481758, -122.226904, 81.0],
 [37.4

In [22]:
tripslat

,Start Date,Start Station,month,day,hour,Id,Lat,Long,count
0,2015-08-31 23:26:00,50,8,0,23,50,37.795392,-122.394203,1
1,2015-08-31 20:53:00,50,8,0,20,50,37.795392,-122.394203,1
2,2015-08-31 20:00:00,50,8,0,20,50,37.795392,-122.394203,1
3,2015-08-31 18:47:00,50,8,0,18,50,37.795392,-122.394203,1
4,2015-08-31 18:25:00,50,8,0,18,50,37.795392,-122.394203,1
...,...,...,...,...,...,...,...,...,...
354147,2014-09-15 17:42:00,25,9,0,17,25,37.486725,-122.225551,1
354148,2014-09-13 17:21:00,25,9,5,17,25,37.486725,-122.225551,1
354149,2014-09-13 17:21:00,25,9,5,17,25,37.486725,-122.225551,1
354150,2014-09-09 15:57:00,25,9,1,15,25,37.486725,-122.225551,1


In [26]:
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format('Bay Area BikeShare Most Popular Stations')

heat_map = folium.Map(location=[37.56236, -122.150876], 
                          tiles='cartodbpositron',
                          zoom_start=10)
heat_map.get_root().html.add_child(folium.Element(title_html));

In [27]:
HeatMap(data=lst, radius=12).add_to(heat_map);

In [28]:
heat_map

In [29]:
df_hour_list = []

for hour in tripslat['hour'].sort_values().unique(): 
    df_hour_list.append(tripslat.loc[tripslat['hour'] == hour,
                                     ['Lat', 'Long', 'count']].groupby(['Lat', 'Long'])
                        .sum().reset_index().values.tolist())

In [41]:
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format('Bay Area BikeShare Station Use by Time of Day')

temp_heat_map = folium.Map(location=[37.56236, -122.150876], 
                          tiles='cartodbpositron',
                          zoom_start=10)
# temp_heat_map.get_root().html.add_child(folium.Element(title_html))

In [42]:
start = datetime.datetime(2021,1,1,0)
end = datetime.datetime(2021,1,1,23)
daterange = pd.date_range(start=start,
                          end=end,
                         periods=24)

time_index = [d.strftime("%I:%M %p") for d in daterange]

HeatMapWithTime(df_hour_list,radius=11,
                index=time_index,
                gradient={0.1: 'blue', 0.5: 'lime', 0.7: 'orange', 1: 'red'}, 
                min_opacity=0.4, 
                max_opacity=0.8, 
                use_local_extrema=True)\
                .add_to(temp_heat_map)

temp_heat_map